# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 3

- Проработать проблематику (в чем смысл для бизнеса)
- Грамотно формализовать задачу
- Проанализировать имеющиеся данные и оценить их пригодность для решения поставленной задачи
- Провести первичный разведочный анализ данных (EDA)

# Preprocessing images

# Импорт библиотек

# Данные

In [ ]:
import os
import pandas as pd
import dill
import tqdm

from sklearn.decomposition import PCA

from PIL import Image

import torch
from torch.utils.data import Dataset

from transformers import CLIPProcessor, CLIPModel

Путь до данных

In [ ]:
data_path = "../../data_closed/"

## Images data

### Чтение данных

Посмотрим для каких айтемов отсутствуют картинки

In [ ]:
with open(data_path + "df_items.dill", "rb") as f:
    df_items = dill.load(f)

In [ ]:
# Sorded list with existing items
item_id_list = sorted(df_items["item_id"].values)

In [ ]:
items_without_img = []

for i in item_id_list:
    if not (os.path.exists(data_path + "images/" + f"{i}.jpg")):
        items_without_img.append(i)

In [ ]:
len(items_without_img)

Всего таких айтемов 269

## CLIP: Zero-shot

In [ ]:
# Получим отсортированный список лейблов изображений,
# для которых присутствуют картиики
labels = sorted(list(set(item_id_list) - set(items_without_img)))

images_paths = [
    data_path + "images/" + f"{idx}.jpg" for idx in labels
]

In [ ]:
# Размер изображения, к которому будем приводить
IMG_SIZE = 128

In [ ]:
# Зададим свой датасет
class CustomImageDataset(Dataset):
    def __init__(
        self,
        file_paths,
        labels,
        transform=None,
        image_size: int= 128,
    ):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.image_size = image_size

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        image = Image.open(self.file_paths[index]).resize(
            size=(self.image_size, self.image_size),
        )
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# Инициализируем датасет
imgs_dataset = CustomImageDataset(
    images_paths[:128  * 20],
    labels=labels[:128  * 20],
    image_size=IMG_SIZE,
)

### Image embeddings

Extract image embeddings on both subsets using pretrained CLIP.

In [ ]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32",
).to(device)
processor = CLIPProcessor.from_pretrained(
    "openai/clip-vit-base-patch32",
)

model.eval();

In [ ]:
def imgs_collate_fn(batch):
    images, labels = zip(*batch)
    images = [
        processor(
            images=image,
            return_tensors="pt",
        )["pixel_values"]
        for image in images
    ]
    images = torch.cat(images, dim=0)
    labels = torch.tensor(labels)
    return images, labels


batch_size = 512

# Get dataloader
imgs_loader = torch.utils.data.DataLoader(
    imgs_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=imgs_collate_fn,
)

In [ ]:
# quick test
batch = next(iter(imgs_loader))
with torch.no_grad():
    out = model.get_image_features(batch[0].to(device))
    lab = batch[1]

display(out.shape)
display(lab.shape)

In [ ]:
def extract_embeddings(
    model, loader, device=device
) -> tuple[torch.Tensor, torch.Tensor]:
    all_embeddings = []
    all_labels = []

    with torch.no_grad():  # Отключаем градиенты для ускорения
        for images, labels in tqdm.tqdm(iter(loader)):
            # Перемещаем данные на указанное устройство
            images = images.to(device)
            labels = labels.to(device)

            # Извлекаем эмбеддинги из модели
            embeddings = model.get_image_features(images)

            # Сохраняем результаты
            all_embeddings.append(embeddings)
            all_labels.append(labels)

    # Объединяем все батчи в один тензор
    all_embeddings = torch.cat(all_embeddings, dim=0).to("cpu")
    all_labels = torch.cat(all_labels, dim=0).to("cpu")

    return all_embeddings, all_labels

In [ ]:
img_embeds, img_labels = extract_embeddings(model, imgs_loader)

In [ ]:
img_labels, img_labels.shape

In [ ]:
img_embeds, img_embeds.shape

## Снизим размерность

In [ ]:
components_to_keep = 25

pca_lowrank = PCA(n_components=components_to_keep)
all_embeddings = pca_lowrank.fit_transform(img_embeds)
all_embeddings.shape
img_embeds_df = pd.concat(
    [
        pd.DataFrame(
            img_labels,
            columns=["item_id"],
        ).reset_index(drop=True),
        pd.DataFrame(
            all_embeddings,
            columns=[f"pca_img_{i}" for i in range(components_to_keep)],
        ).reset_index(drop=True),
    ],
    axis=1,
)

img_embeds_df.head()